<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVIS-CAValli_Team/blob/main/Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') # turned out to be a good workaround to load a huge amount of data and keep it available

Mounted at /content/drive


# Load tree_dataset

In [24]:
tree_dataset = pd.read_csv("/content/drive/MyDrive/tree_dataset/final_dataset/tree_dataset.csv", low_memory=False)

# Prepare data for mapping

In [3]:
def create_abundance_csv(csv_name, header, name, abundance):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx]]
      writer.writerow(row)

In [4]:
def create_abundance_fips_csv(csv_name, header, name, abundance, fips_codes):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx], fips_codes.get(name[idx])]
      writer.writerow(row)

## State abundance

In [5]:
csv_name = "state_abundance.csv"
header = ['state', 'abundance']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_csv(csv_name, header, state, abundance)

## State abundance and fips code

In [6]:
fips_codes = {
    "Alabama": "US01",
    "Alaska": "US02",
    "Arizona": "US04",
    "Arkansas": "US05",
    "California": "US06",
    "Colorado": "US08",
    "Connecticut": "US09",
    "Delaware": "US10",
    "District of Columbia": "US11",
    "Florida": "US12",
    "Georgia": "US13",
    "Hawaii": "US15",
    "Idaho": "US16",
    "Illinois": "US17",
    "Indiana": "US18",
    "Iowa": "US19",
    "Kansas": "US20",
    "Kentucky": "US21",
    "Louisiana": "US22",
    "Maine": "US23",
    "Maryland": "US24",
    "Massachussetts": "US25",
    "Michigan": "US26",
    "Minnesota": "US27",
    "Mississippi": "US28",
    "Missouri": "US29",
    "Montana": "US30",
    "Nebraska": "US31",
    "Nevada": "US32",
    "New Hampshire": "US33",
    "New Jersey": "US34",
    "New Mexico": "US35",
    "New York": "US36",
    "North Carolina": "US37",
    "North Dakota": "US38",
    "Ohio": "US39",
    "Oklahoma": "US40",
    "Oregon": "US41",
    "Pennsylvania": "US42",
    "Rhode Island": "US44",
    "South Carolina": "US45",
    "South Dakota": "US46",
    "Tennessee": "US47",
    "Texas": "US48",
    "Utah": "US49",
    "Vermont": "US50",
    "Virginia": "US51",
    "Washington": "US53",
    "West Virginia": "US54",
    "Wisconsin": "US55",
    "Wyoming": "US56"
}

In [7]:
csv_name = "state_abundance_fips.csv"
header = ['state', 'abundance', 'fips']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, fips_codes)

## State abundance and postal

In [8]:
postal_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [9]:
csv_name = "state_abundance_postal.csv"
header = ['state', 'abundance', 'postal']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, postal_codes)

## City abundance

In [10]:
csv_name = "city_abundance.csv"
header = ['city', 'abundance']
city = list(tree_dataset['greater_metro'].value_counts().index)
abundance = list(tree_dataset['greater_metro'].value_counts().values)
create_abundance_csv(csv_name, header, city, abundance)

## JSON

In [11]:
abundance_df = pd.read_csv("/content/state_abundance_postal.csv", low_memory=False)
abundance_dict = {}

for idx in range(abundance_df.shape[0]):
  name = list(abundance_df.iloc[idx])[0]
  abundance = list(abundance_df.iloc[idx])[1]
  postal = list(abundance_df.iloc[idx])[2]
  abundance_dict[postal] = [name, abundance]

for key, value in postal_codes.items():
  if abundance_dict.get(value) is None:
    name = key
    abundance = 0
    abundance_dict[value] = [name, abundance]
  else:
    name = abundance_dict.get(value)[0]
    abundance = abundance_dict.get(value)[1]

In [12]:
json_path = "/content/drive/MyDrive/tree_dataset/final_dataset/us-states.json"
with open(json_path, 'r') as file:
    json_file = json.load(file)

In [13]:
# import geopandas
# data = geopandas.read_file(json_path)['geometry']
# data.crs = 'EPSG:4326'
# state_polygons = geopandas.read_file(json_path)[['postal', 'geometry']]
# state_polygons['area'] = data.to_crs('EPSG:3857').geometry.area / (10**6)
# print(state_polygons[['postal', 'area']])

In [14]:
areas_dict = {
    'Alabama': 135767,
    'Alaska': 1717855,
    'Arizona': 295234,
    'Arkansas': 137733,
    'California': 423968,
    'Colorado': 269601,
    'Connecticut': 12534,
    'Delaware': 5047,
    'District of Columbia': 177,
    'Florida': 170304,
    'Georgia': 148958,
    'Hawaii': 16637,
    'Idaho': 216443,
    'Illinois': 144774,
    'Indiana': 92789,
    'Iowa': 145746,
    'Kansas': 211754,
    'Kentucky': 102908,
    'Louisiana': 135659,
    'Maine': 91646,
    'Maryland': 25313,
    'Massachusetts': 20203,
    'Michigan': 250488,
    'Minnesota': 225163,
    'Mississippi': 120383,
    'Missouri': 180560,
    'Montana': 380832,
    'Nebraska': 199097,
    'Nevada': 286380,
    'New Hampshire': 23186,
    'New Jersey': 19050,
    'New Mexico': 314917,
    'New York': 141297,
    'North Carolina': 139390,
    'North Dakota': 178712,
    'Ohio': 116096,
    'Oklahoma': 177846,
    'Oregon': 248608,
    'Pennsylvania': 119283,
    'Rhode Island': 2678,
    'South Carolina': 82931,
    'South Dakota': 199729,
    'Tennessee': 109153,
    'Texas': 695662,
    'Utah': 219882,
    'Vermont': 24906,
    'Virginia': 110787,
    'Washington': 184666,
    'West Virginia': 62259,
    'Wisconsin': 140268,
    'Wyoming': 253335
}

In [15]:
for idx in range(len(json_file['objects']['states'].get('geometries'))):
  properties = json_file['objects']['states'].get('geometries')[idx]['properties']
  postal = properties['postal']
  try:
    name, abundance = abundance_dict[postal]
    area = areas_dict[name]
    properties['name'] = name
    properties['abundance'] = str(abundance)
    properties['area'] = area
    properties['density'] = round(abundance / area, 5)
    properties['density_1000'] = round(abundance / area * 1000, 2)
    print(properties)
  except:
    pass

{'postal': 'AK', 'name': 'Alaska', 'abundance': '0', 'area': 1717855, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AL', 'name': 'Alabama', 'abundance': '0', 'area': 135767, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AR', 'name': 'Arkansas', 'abundance': '0', 'area': 137733, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'AZ', 'name': 'Arizona', 'abundance': '214', 'area': 295234, 'density': 0.00072, 'density_1000': 0.72}
{'postal': 'CA', 'name': 'California', 'abundance': '1806085', 'area': 423968, 'density': 4.25996, 'density_1000': 4259.96}
{'postal': 'CO', 'name': 'Colorado', 'abundance': '354878', 'area': 269601, 'density': 1.31631, 'density_1000': 1316.31}
{'postal': 'CT', 'name': 'Connecticut', 'abundance': '0', 'area': 12534, 'density': 0.0, 'density_1000': 0.0}
{'postal': 'DC', 'name': 'District of Columbia', 'abundance': '175847', 'area': 177, 'density': 993.48588, 'density_1000': 993485.88}
{'postal': 'DE', 'name': 'Delaware', 'abundance': '0', 'area': 5047, 'densi

In [16]:
out_path = "/content/choropleth.json"
with open(out_path, 'w') as file:
    json.dump(json_file, file)

## Drop data with no longitude/latitude

In [25]:
tree_dataset.dropna(subset=['longitude_coordinate', 'latitude_coordinate'], how='any', inplace=True)

### Prepare dataset (just one class)

In [26]:
dotmap_data = tree_dataset[['state', 'longitude_coordinate', 'latitude_coordinate']]
dotmap_data.to_csv('dotmap_full.csv', index=False)

In [27]:
dotmap_data['longitude_coordinate_rounded'] = round(dotmap_data['longitude_coordinate'], 1)
dotmap_data['latitude_coordinate_rounded'] = round(dotmap_data['latitude_coordinate'], 1)

<ipython-input-27-09e52983d8c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotmap_data['longitude_coordinate_rounded'] = round(dotmap_data['longitude_coordinate'], 1)
<ipython-input-27-09e52983d8c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotmap_data['latitude_coordinate_rounded'] = round(dotmap_data['latitude_coordinate'], 1)


In [29]:
dotmap_rounded = dotmap_data[['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded']
                             ].value_counts().reset_index().sort_values(
                                 by = ['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded'])
dotmap_rounded.columns = ['state', 'longitude', 'latitude', 'count']

In [30]:
dotmap_rounded

,state,longitude,latitude,count
490,California,-122.8,38.5,95
278,California,-122.7,38.5,1118
71,California,-122.5,37.7,20672
60,California,-122.5,37.8,26481
672,California,-122.4,37.5,1
...,...,...,...,...
203,Wisconsin,-87.9,43.0,3012
186,Wisconsin,-87.9,43.1,3869
276,Wisconsin,-87.9,43.2,1123
360,Wisconsin,-87.8,42.9,484


In [33]:
dotmap_rounded2 = dotmap_data[['state', 'longitude_coordinate_rounded', 'latitude_coordinate_rounded']
                              ].value_counts().reset_index().sort_values(
                                  by = ['longitude_coordinate_rounded', 'latitude_coordinate_rounded', 'state'])
dotmap_rounded2.columns = ['state', 'longitude', 'latitude', 'count']

GUARDARE QUI PER ELIMINARE

In [35]:
dotmap_rounded2
### Eliminare pag 8-11 --> tutte le occorrenze California dal 588 (sotto New Mexico) in poi
### Eliminare pag 12-15 --> tutte le occorrenze Minnesota dal 472 (sotto Oklahoma) in poi (fino a sopra Texas)
### Eliminare pag 18-19 --> tutte le occorrenze Texas dal 630 (sopra Iowa) in poi


,state,longitude,latitude,count
536,Hawaii,-158.0,21.4,43
264,Hawaii,-157.9,21.3,1362
680,Hawaii,-157.9,21.4,1
153,Hawaii,-157.8,21.3,6224
200,Hawaii,-157.8,21.4,3122
...,...,...,...,...
655,California,-72.2,42.0,3
247,Rhode Island,-71.5,41.8,1562
502,Rhode Island,-71.5,41.9,76
80,Rhode Island,-71.4,41.8,19266


In [20]:
dotmap_rounded.to_csv('dotmap.csv', index=False)

# Download ALL .csv files

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname=os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>